Create the Collection

In [27]:
from qdrant_client import QdrantClient
from qdrant_client.http.models import VectorParams

# Initialize Qdrant Client
qdrant_client = QdrantClient(
    url="2040fc5f-bf8d-4921-8213-075eee2403f1.europe-west3-0.gcp.cloud.qdrant.io:6333",  # Replace with your Qdrant cluster URL
    api_key="13NTI6NfrXXG_oE28Re5bpqPvyrGu1tXG_SIM_GnSS28zghRkeMAOQ"  # Replace with your API key
)

# Define the collection name
collection_name = "semantic_search"

# Define vectors configuration
vectors_config = {
    "default": VectorParams(
        size=768,  # Number of dimensions in your vectors
        distance="Cosine"  # The distance metric to use
    )
}

# Check if the collection exists
if not qdrant_client.collection_exists(collection_name):
    # Create the collection if it doesn't exist
    qdrant_client.create_collection(
        collection_name=collection_name,
        vectors_config=vectors_config
    )
else:
    print(f"Collection '{collection_name}' already exists.")


Collection 'semantic_search' already exists.


In [9]:
# Delete the existing collection (if you want to start fresh)
qdrant_client.delete_collection(collection_name)

# Recreate the collection with the correct vector size
vectors_config = {
    "default": VectorParams(
        size=384,  # Adjust the vector size here
        distance="Cosine"
    )
}

qdrant_client.create_collection(
    collection_name=collection_name,
    vectors_config=vectors_config
)

True

Insert Vectors into Qdrant

In [11]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Load a pre-trained model that outputs 768-dimensional vectors
model = SentenceTransformer('all-MiniLM-L6-v2')

# Example documents to be inserted
documents = [
    "This is the first document.",
    "This document is the second one.",
    "And here is the third document."
]

# Convert documents to vectors
vectors = model.encode(documents)

# Prepare payload (optional metadata for each vector)
payloads = [{"id": i, "text": doc} for i, doc in enumerate(documents)]

# Insert vectors into Qdrant collection
for i, vector in enumerate(vectors):
    qdrant_client.upsert(
        collection_name=collection_name,
        points=[
            {
                "id": i,  # unique identifier for the vector
                "vector": {"default": vector.tolist()},  # specify the vector name as "default"
                "payload": payloads[i]  # optional metadata
            }
        ]
    )

print("Vectors inserted successfully.")


Vectors inserted successfully.


Update a Vector

In [15]:
# Example: Update the first document
updated_document = "This is the updated first document."
updated_vector = model.encode([updated_document])[0]

qdrant_client.upsert(
    collection_name=collection_name,
    points=[
        {
            "id": 0,  # ID of the vector to update
            "vector": {"default": updated_vector.tolist()},
            "payload": {"id": 0, "text": updated_document}
        }
    ]
)

print("Vector updated successfully.")


Vector updated successfully.


Delete a Vector

In [21]:
from qdrant_client.http import models as qmodels

# Example: Delete the vector with ID 1
qdrant_client.delete(
    collection_name=collection_name,
    points_selector=qmodels.PointIdsList(
        points=[1]  # Replace with the ID of the vector you want to delete
    )
)

print("Vector deleted successfully.")


Vector deleted successfully.
